In [5]:
import pandas as pd
import cv2
import os
from imutils import build_montages
import tqdm

In [56]:
def bestByImg(csvPath):
    df = pd.read_csv(csvPath)
    df['conf'] = df.obj_conf * df.cls_conf

    # Sort the DataFrame by 'conf' column in descending order
    # Use groupby to get the rows with the maximum 'conf' value for each unique 'img'
    result = (df[df.cls_conf!=1]
        .sort_values(by='conf', ascending=False)
        .groupby('img').first().reset_index())

    return result

# def truePositives(df):


def buildMontage(df, imgRoot, show=False):
    images = []

    for imgPath in tqdm.tqdm(df.img):
        image = cv2.imread(os.path.join(imgRoot, imgPath.replace('txt', 'jpg')))
        imgRow = df[df['img'] == imgPath].first()
        # print(int(xmin))
        xmin = int(imgRow.xmin)
        ymin = int(imgRow.ymin)
        xmax = int(imgRow.xmax)
        ymax = int(imgRow.ymax)
        image = cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (0,255,0), 3)
        images.append(image)

    montages = build_montages(images, (640//3, 640//4), (5,5))

    if show:
        showMontages(montages)

    return montages

def showMontages(montages):
    for montage in montages:
        cv2.imshow('montage', montage)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [57]:
csvPath = 'fg_df_detections.csv'
topConf_df = bestByImg(csvPath=csvPath)
topConf_df

,img,class,cls_conf,obj_conf,xmin,xmax,ymin,ymax,conf
0,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.970897,0.025770,472,566,209,303,0.025020
1,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.998580,0.377683,374,428,62,117,0.377147
2,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.998500,0.617392,389,440,93,144,0.616466
3,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.998508,0.623156,389,440,92,144,0.622226
4,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.998592,0.454227,388,442,101,155,0.453587
...,...,...,...,...,...,...,...,...,...
728,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.997652,0.136882,559,607,40,88,0.136561
729,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,1,0.262368,0.424114,245,292,239,286,0.111274
730,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.592576,0.154289,255,300,246,291,0.091428
731,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.999049,0.610919,415,459,92,135,0.610338


In [58]:
df = pd.read_csv(csvPath)

In [59]:
labels_df = df[df.cls_conf==1].reset_index(drop=True)
labels_s_df = labels_df.drop_duplicates(subset=['img'], keep=False, ignore_index=True)
labels_s_df

,img,class,cls_conf,obj_conf,xmin,xmax,ymin,ymax
0,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,1.0,1.0,467,557,210,300
1,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,1.0,1.0,376,428,65,117
2,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,1.0,1.0,387,439,91,143
3,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,1.0,1.0,387,439,91,143
4,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,1.0,1.0,388,446,97,155
...,...,...,...,...,...,...,...,...
520,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,1,1.0,1.0,446,514,92,160
521,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,9,1.0,1.0,242,294,238,290
522,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,9,1.0,1.0,256,302,245,291
523,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,5,1.0,1.0,416,460,92,136


In [60]:
result_df = labels_s_df.merge(df[df.cls_conf!=1], on=['img', 'class'], how='inner', suffixes=('','_y'))
result_df

,img,class,cls_conf,obj_conf,xmin,xmax,ymin,ymax,cls_conf_y,obj_conf_y,xmin_y,xmax_y,ymin_y,ymax_y
0,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,1.0,1.0,388,446,97,155,0.001626,0.669604,21,95,144,217
1,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,1.0,1.0,388,446,97,155,0.001631,0.676230,21,95,143,217
2,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,1.0,1.0,571,633,164,226,0.102770,0.053737,484,537,175,227
3,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,1.0,1.0,571,633,164,226,0.100783,0.050626,484,536,175,227
4,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,5,1.0,1.0,477,519,155,197,0.002143,0.589845,399,524,376,501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,5,1.0,1.0,212,392,261,441,0.003004,0.389098,140,193,155,207
373,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,5,1.0,1.0,518,582,8,72,0.003814,0.352302,390,476,109,196
374,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,1,1.0,1.0,437,515,87,165,0.009782,0.113258,564,605,143,185
375,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,1,1.0,1.0,446,514,92,160,0.297796,0.019029,563,612,150,200


In [61]:
imgRoot = "C:\\Users\\leofi\OneDrive - Universidade de Lisboa\Documents\GitHub\masters\Data\\fc2015_yolo_uc_hard/test/images"
montages = buildMontage(result_df, imgRoot=imgRoot, show=True)

  0%|          | 0/377 [00:00<?, ?it/s]


TypeError: NDFrame.first() missing 1 required positional argument: 'offset'